#### Reinforcement Learning and Decision Making &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
# &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The $\lambda$ Return

## Description

Given an MDP and a particular time step $t$ of a task (continuing or
episodic), the $\lambda$-return, $G_t^\lambda$, $0\leq\lambda\leq 1$, is
a weighted combination of the $n$-step returns $G_{t:t+n}$, $n \geq 1$:

$${G_t^\lambda = \sum\limits_{n=1}^\infty(1-\lambda)\lambda^{n-1}G_{t:t+n}.}$$

While the $n$-step return $G_{t:t+n}$ can be viewed as the target of
an $n$-step TD update rule, the $\lambda$-return can be viewed as the
target of the update rule for the TD$(\lambda)$ prediction algorithm.

Consider the Markov reward process described by the following state
diagram and assume the agent is in state $0$ at time $t$ (also assume
the discount rate is $\gamma=1$). A Markov reward process can be thought of as
an MDP with only one action possible from each state (denoted as action $0$ in
the figure below).

![image](https://d1b10bmlvqabco.cloudfront.net/paste/jqmfo7d3watba/9e6fd83672f880704b8418728297fc077786c8907d87fec631601da9ff4c85ef/hw2.png)

## Procedure

-   You will implement your solution using the `solve()` method
    in the code below.
    
-   You will be given `p`, the probability of transitioning from state
    $0$ to state $1$, `V`, the estimate of the value function at time
    $t$, represented as a vector
    $[V(0), V(1), V(2), V(3), V(4), V(5), V(6)]$, and `rewards`, a
    vector of the rewards $[r_0, r_1, r_2, r_3, r_4, r_5, r_6]$
    corresponding to the MDP.

-   Your return value should be a value of $\lambda$,
    strictly less than 1, such that the expected value of the
    $\lambda$-return equals the expected Monte-Carlo return at time $t$.

-   Your answer must be correct to $3$ decimal places, truncated (e.g.
    3.14159265 becomes 3.141).

## Resources

The concepts explored in this homework are covered by:

-   Lecture Lesson 3: TD and Friends

-   Chapter 7 (7.1 $n$-step TD Prediction) and Chapter 12 (12.1 The
    $\lambda$-return) of
    http://incompleteideas.net/book/the-book-2nd.html

-   'Learning to Predict by the Method of Temporal Differences', R.
    Sutton, 1988



In [1]:
################
# Versions
# numpy==1.18.0
################
import numpy as np

class TDAgent(object):
    def __init__(self):
        pass
       
    def get_expec_value(self):
        expec_value = [0]* len(self.state_values)
        expec_rewards = [0]* len(self.state_values)
        one_minus_proba = 1-self.proba
        # expec_value[0] = self.state_values[0]
        expec_rewards[1] = self.proba * self.state_returns[0] + one_minus_proba * self.state_returns[1]
        expec_value[1] = self.proba * self.state_values[1] + one_minus_proba * self.state_values[2] + expec_rewards[1]
        expec_rewards[2] = self.proba*(self.state_returns[0] + self.state_returns[2]) + one_minus_proba * (self.state_returns[1]+self.state_returns[3])
        expec_value[2] =  self.state_values[3] + expec_rewards[2]
        for state_num in range(3 , 6): # from 3 to last state: 5
            expec_rewards[state_num] =  expec_rewards[state_num -1] + self.state_returns[state_num + 1 ]
            expec_value[state_num ] =  expec_rewards[state_num ] + self.state_values[state_num +1 ]
        return expec_value

    def get_error(self, lambda_value, expec_value):
        error = 0
        one_minus_lbd = 1 - lambda_value
        for k in range(1,5): # from 1 to 4
            error += lambda_value **(k - 1) * expec_value[k]
        error = error *  one_minus_lbd
        error += (lambda_value**4-1) * expec_value[5]
        return error

    def solve(self,p,V,rewards):
        self.proba= p
        self.state_values = V
        self.state_returns= rewards
        try_lambda = 0.5  # initial value of lambda
        thera = 0.002
        step = 0.1
        expec_value = self.get_expec_value()
        last_sign_dirt = 1
        while True:
            error_now = self.get_error( try_lambda,expec_value)
            grat = (self.get_error(try_lambda + step,expec_value) - error_now)/step
            sign_dirt = np.sign(grat) * np.sign(error_now)
            if abs(error_now) < thera:
                return np.round(try_lambda,3)
            else:
                if (sign_dirt != last_sign_dirt):
                    step = 0.5 * step
                try_lambda += -step* sign_dirt                  
                last_sign_dirt = sign_dirt


## Test cases

We have provided some test cases for you to help verify your implementation.

In [2]:
## DO NOT MODIFY THIS CODE.  This code will ensure that your submission
## will work proberly with the autograder

import unittest

class TestTDNotebook(unittest.TestCase):
    def test_case_1(self):
        agent = TDAgent()
        np.testing.assert_almost_equal(
            agent.solve(
                p=0.81,
                V=[0.0, 4.0, 25.7, 0.0, 20.1, 12.2, 0.0],
                rewards=[7.9, -5.1, 2.5, -7.2, 9.0, 0.0, 1.6]
            ),
            0.622,
            decimal=3
        )
        
    def test_case_2(self):
        agent = TDAgent()
        np.testing.assert_almost_equal(
            agent.solve(
                p=0.22,
                V=[12.3, -5.2, 0.0, 25.4, 10.6, 9.2, 0.0],
                rewards=[-2.4, 0.8, 4.0, 2.5, 8.6, -6.4, 6.1]
            ),
            0.519,
            decimal=3
        )
        
    def test_case_3(self):
        agent = TDAgent()

        np.testing.assert_almost_equal(
            agent.solve(
                p=0.64,
                V=[-6.5, 4.9, 7.8, -2.3, 25.5, -10.2, 0.0],
                rewards=[-2.4, 9.6, -7.8, 0.1, 3.4, -2.1, 7.9]
            ),
            0.207,
            decimal=3
        )

unittest.main(argv=[''], verbosity=2, exit=False)

test_case_1 (__main__.TestTDNotebook) ... ok
test_case_2 (__main__.TestTDNotebook) ... ok
test_case_3 (__main__.TestTDNotebook) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.010s

OK
